In [11]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MyDrive


In [12]:
!ls -l /content/drive/MyDrive/

total 1377
-rw------- 1 root root 1392695 Aug 18 05:38 a.xlsx
drwx------ 2 root root    4096 Feb 20  2022 Classroom
drwx------ 4 root root    4096 Oct 13 06:25 DeepL
drwx------ 2 root root    4096 Nov 11  2022 RCSC
drwx------ 2 root root    4096 Oct 25  2022 SEAGULL
-rw------- 1 root root     171 Jul 31 05:35 無題のドキュメント.gdoc


In [13]:
!ls -l /content/drive/MyDrive/DeepL/Img-Bg-White

total 93
-rw------- 1 root root 57878 Oct 13 07:35 1.jpg
-rw------- 1 root root     0 May 16 04:33 app.py
-rw------- 1 root root   147 May 16 04:33 config.py
-rw------- 1 root root  8772 May 16 04:33 data_loader.py
-rw------- 1 root root  3502 Oct 13 07:40 __init__.py
drwx------ 3 root root  4096 Oct 13 06:30 model
-rw------- 1 root root  1035 May 16 04:33 README.md
-rw------- 1 root root   480 May 16 04:33 requirements.txt
drwx------ 3 root root  4096 Oct 13 06:30 saved_models
drwx------ 5 root root  4096 Oct 13 06:30 static
-rw------- 1 root root  8447 Oct 13 07:36 Untitleaaa.ipynb


In [14]:
%cd /content/drive/MyDrive/DeepL/Img-Bg-White

/content/drive/MyDrive/DeepL/Img-Bg-White


In [15]:
%run __init__.py

---Loading Model---
---Removing Background...
---Success---


THIS ONE WILL NOT WORK

In [ ]:
%mkdir saved_models
%mkdir saved_models/u2net

mkdir: cannot create directory ‘saved_models’: File exists
mkdir: cannot create directory ‘saved_models/u2net’: File exists


In [ ]:
path_proj = '/content/drive/MyDrive/DeepDark/Img-Bg-Remover-Wk'

with open(path_proj) as f:


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import uuid
import os

from model import U2NET
from torch.autograd import Variable
from skimage import io, transform
from PIL import Image

# Get The Current Directory
currentDir = os.path.dirname(__file__)

# Functions:
# Save Results


def save_output(image_name, output_name, pred, d_dir, type):
    predict = pred
    predict = predict.squeeze()
    predict_np = predict.cpu().data.numpy()
    im = Image.fromarray(predict_np*255).convert('RGB')
    image = io.imread(image_name)
    imo = im.resize((image.shape[1], image.shape[0]))
    pb_np = np.array(imo)
    if type == 'image':
        # Make and apply mask
        mask = pb_np[:, :, 0]
        mask = np.expand_dims(mask, axis=2)
        imo = np.concatenate((image, mask), axis=2)
        imo = Image.fromarray(imo, 'RGBA')

    imo.save(d_dir+output_name)
# Remove Background From Image (Generate Mask, and Final Results)


def removeBg(imagePath):
    inputs_dir = os.path.join(currentDir, 'static/inputs/')
    results_dir = os.path.join(currentDir, 'static/results/')
    masks_dir = os.path.join(currentDir, 'static/masks/')

    # convert string of image data to uint8
    with open(imagePath, "rb") as image:
        f = image.read()
        img = bytearray(f)

    nparr = np.frombuffer(img, np.uint8)

    if len(nparr) == 0:
        return '---Empty image---'

    # decode image
    try:
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    except:
        # build a response dict to send back to client
        return "---Empty image---"

    # save image to inputs
    unique_filename = str(uuid.uuid4())
    cv2.imwrite(inputs_dir+unique_filename+'.jpg', img)

    # processing
    image = transform.resize(img, (320, 320), mode='constant')

    tmpImg = np.zeros((image.shape[0], image.shape[1], 3))

    tmpImg[:, :, 0] = (image[:, :, 0]-0.485)/0.229
    tmpImg[:, :, 1] = (image[:, :, 1]-0.456)/0.224
    tmpImg[:, :, 2] = (image[:, :, 2]-0.406)/0.225

    tmpImg = tmpImg.transpose((2, 0, 1))
    tmpImg = np.expand_dims(tmpImg, 0)
    image = torch.from_numpy(tmpImg)

    image = image.type(torch.FloatTensor)
    image = Variable(image)

    d1, d2, d3, d4, d5, d6, d7 = net(image)
    pred = d1[:, 0, :, :]
    ma = torch.max(pred)
    mi = torch.min(pred)
    dn = (pred-mi)/(ma-mi)
    pred = dn

    save_output(inputs_dir+unique_filename+'.jpg', unique_filename +
                '.png', pred, results_dir, 'image')
    save_output(inputs_dir+unique_filename+'.jpg', unique_filename +
                '.png', pred, masks_dir, 'mask')
    return "---Success---"


# ------- Load Trained Model --------
print("---Loading Model---")
model_name = 'u2net'
model_dir = os.path.join(currentDir, 'saved_models',
                         model_name, model_name + '.pth')
net = U2NET(3, 1)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(model_dir))
    net.cuda()
else:
    net.load_state_dict(torch.load(model_dir, map_location='cpu'))
# ------- Load Trained Model --------


print("---Removing Background...")
# ------- Call The removeBg Function --------
imgPath = "1.jpg"  # Change this to your image path
print(removeBg(imgPath))
